In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Try known uploaded path first, then fallback to local file name
possible_paths = ['/mnt/data/train.csv', 'train.csv']
for p in possible_paths:
    if os.path.exists(p):
        data_path = p
        break
else:
    raise FileNotFoundError("train.csv not found in /mnt/data or current directory.")

df = pd.read_csv(data_path)
df_original = df.copy()   # keep an untouched copy
print("Loaded file:", data_path)
print("Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())


Loaded file: train.csv
Shape: (891, 12)

First 5 rows:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  


In [3]:
# 1) Show missing values per column
print("Missing values (count) per column:")
print(df.isnull().sum())

# 2) Drop columns with >50% missing values
missing_frac = df.isnull().mean()
cols_to_drop = missing_frac[missing_frac > 0.5].index.tolist()
print("\nColumns with >50% missing (will drop):", cols_to_drop)
df.drop(columns=cols_to_drop, inplace=True)

# 3) Fill missing numerical values with median
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("\nNumerical columns detected:", num_cols)
for col in num_cols:
    if df[col].isnull().sum() > 0:
        med = df[col].median()
        df[col].fillna(med, inplace=True)
        print(f"Filled missing numeric column '{col}' with median = {med}")

# 4) Fill missing categorical values with mode
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print("\nCategorical columns detected:", cat_cols)
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)
        print(f"Filled missing categorical column '{col}' with mode = '{mode_val}'")

print("\nMissing values after imputation:")
print(df.isnull().sum())


Missing values (count) per column:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

Columns with >50% missing (will drop): []

Numerical columns detected: ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

Categorical columns detected: ['Name', 'Sex', 'Ticket', 'Embarked']

Missing values after imputation:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [ ]:
dup_before = df.duplicated().sum()
print("Duplicate rows found before removal:", dup_before)

if dup_before > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicates removed.")
else:
    print("No duplicate rows found.")

dup_after = df.duplicated().sum()
print("Duplicate rows after removal:", dup_after)
print("Shape after duplicate removal:", df.shape)


In [4]:
# Optional: drop very high-cardinality textual fields (uncomment if desired)
for col in ['Name','Ticket']:
    if col in df.columns:
        print(f"NOTE: Column '{col}' is text-heavy. Consider dropping it for modeling.")
        # df.drop(columns=[col], inplace=True)   # <- uncomment to drop

# Convert boolean dtype columns to 0/1
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)
        print(f"Converted boolean column '{col}' to int (0/1).")

# Identify categorical columns to one-hot encode
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print("Columns to one-hot encode:", cat_cols)

# Apply one-hot encoding (creates new columns and drops the originals)
df = pd.get_dummies(df, columns=cat_cols, drop_first=False)
print("Shape after one-hot encoding:", df.shape)

# If you want to see a few columns after encoding:
print("\nColumns sample after encoding:", list(df.columns)[:40])


NOTE: Column 'Name' is text-heavy. Consider dropping it for modeling.
NOTE: Column 'Ticket' is text-heavy. Consider dropping it for modeling.
Columns to one-hot encode: ['Name', 'Sex', 'Ticket', 'Embarked']
Shape after one-hot encoding: (891, 1584)

Columns sample after encoding: ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Name_Abbing, Mr. Anthony', 'Name_Abbott, Mr. Rossmore Edward', 'Name_Abbott, Mrs. Stanton (Rosa Hunt)', 'Name_Abelson, Mr. Samuel', 'Name_Abelson, Mrs. Samuel (Hannah Wizosky)', 'Name_Adahl, Mr. Mauritz Nils Martin', 'Name_Adams, Mr. John', 'Name_Ahlin, Mrs. Johan (Johanna Persdotter Larsson)', 'Name_Aks, Mrs. Sam (Leah Rosen)', 'Name_Albimona, Mr. Nassef Cassem', 'Name_Alexander, Mr. William', 'Name_Alhomaki, Mr. Ilmari Rudolf', 'Name_Ali, Mr. Ahmed', 'Name_Ali, Mr. William', 'Name_Allen, Miss. Elisabeth Walton', 'Name_Allen, Mr. William Henry', 'Name_Allison, Master. Hudson Trevor', 'Name_Allison, Miss. Helen Loraine', 'Name_Allison, Mrs

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# determine numeric columns excluding target/id
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in ['PassengerId','Survived']:
    if c in numeric_cols:
        numeric_cols.remove(c)
print("Numeric columns to scale:", numeric_cols)

# 1) Min-Max scaling (0-1)
minmax_scaler = MinMaxScaler()
df_minmax = df.copy()
df_minmax[numeric_cols] = minmax_scaler.fit_transform(df_minmax[numeric_cols])
print("\nMin-Max scaling applied. Summary (min, max, mean, std):")
print(df_minmax[numeric_cols].describe().T[['min','max','mean','std']])

# 2) StandardScaler (mean=0, std=1)
std_scaler = StandardScaler()
df_standard = df.copy()
df_standard[numeric_cols] = std_scaler.fit_transform(df_standard[numeric_cols])
print("\nStandard scaling applied. Summary (min, max, mean, std):")
print(df_standard[numeric_cols].describe().T[['min','max','mean','std']])


In [5]:
# We'll detect outliers on numeric_cols (the same list used above) and replace them with the median.
df_outliers_handled = df.copy()   # start from encoded + imputed dataset

# Ensure numeric_cols list excludes target/id if present
numeric_cols = df_outliers_handled.select_dtypes(include=[np.number]).columns.tolist()
for c in ['PassengerId','Survived']:
    if c in numeric_cols:
        numeric_cols.remove(c)

outlier_summary = {}
for col in numeric_cols:
    Q1 = df_outliers_handled[col].quantile(0.25)
    Q3 = df_outliers_handled[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    mask = (df_outliers_handled[col] < lower) | (df_outliers_handled[col] > upper)
    cnt = int(mask.sum())
    outlier_summary[col] = cnt
    if cnt > 0:
        med = df_outliers_handled[col].median()
        df_outliers_handled.loc[mask, col] = med
        print(f"Column '{col}': {cnt} outliers replaced with median = {med}")
    else:
        print(f"Column '{col}': No outliers detected")

print("\nOutliers count per column (before replacement):")
print(outlier_summary)


Column 'Pclass': No outliers detected
Column 'Age': 66 outliers replaced with median = 28.0
Column 'SibSp': 46 outliers replaced with median = 0.0
Column 'Parch': 213 outliers replaced with median = 0.0
Column 'Fare': 116 outliers replaced with median = 14.4542

Outliers count per column (before replacement):
{'Pclass': 0, 'Age': 66, 'SibSp': 46, 'Parch': 213, 'Fare': 116}


In [6]:

# ===== Final: Scale again (optional) and Save cleaned datasets =====
# If you want the final saved datasets with scaling applied after outlier handling, create them:

# numeric columns to scale (same logic as before)
final_numeric_cols = df_outliers_handled.select_dtypes(include=[np.number]).columns.tolist()
for c in ['PassengerId','Survived']:
    if c in final_numeric_cols:
        final_numeric_cols.remove(c)

# MinMax scaled final
final_minmax = df_outliers_handled.copy()
final_minmax[final_numeric_cols] = MinMaxScaler().fit_transform(final_minmax[final_numeric_cols])

# Standard scaled final
final_standard = df_outliers_handled.copy()
final_standard[final_numeric_cols] = StandardScaler().fit_transform(final_standard[final_numeric_cols])

# Save CSVs (these files will be saved in the current working directory or notebook workspace)
final_files = {
    'cleaned_no_scaling.csv': df_outliers_handled,
    'cleaned_minmax.csv': final_minmax,
    'cleaned_standard.csv': final_standard
}
for fname, dataframe in final_files.items():
    dataframe.to_csv(fname, index=False)
    print(f"Saved: {fname} (shape: {dataframe.shape})")

print("\nDone. Three cleaned files saved: cleaned_no_scaling.csv, cleaned_minmax.csv, cleaned_standard.csv")


Saved: cleaned_no_scaling.csv (shape: (891, 1584))
Saved: cleaned_minmax.csv (shape: (891, 1584))
Saved: cleaned_standard.csv (shape: (891, 1584))

Done. Three cleaned files saved: cleaned_no_scaling.csv, cleaned_minmax.csv, cleaned_standard.csv
